## Homework - Monitoring and Evaluating RAGs

Link [Homework 4](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/04-monitoring/homework.md)

### Dataset Prep

In [38]:
%pip install pandas
%pip install sentence-transformers
%pip install tqdm
%pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [8]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

url = f'{github_url}?raw=1'
df = pd.read_csv(url)

df = df.iloc[:300]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   answer_llm   300 non-null    object
 1   answer_orig  300 non-null    object
 2   document     300 non-null    object
 3   question     300 non-null    object
 4   course       300 non-null    object
dtypes: object(5)
memory usage: 11.8+ KB


## Q1

In [17]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'

embedding_model = SentenceTransformer(model_name)

In [16]:
answer_llm = df.iloc[0].answer_llm
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [18]:
answer_encoding = embedding_model.encode(answer_llm)

answer_encoding[0]


np.float32(-0.42244655)

## Q2

In [22]:
from tqdm.auto import tqdm
import numpy as np

In [21]:
evaluations = []

for idx, row in tqdm(df.iterrows()):
    original_vector = embedding_model.encode(row['answer_orig'])
    llm_vector = embedding_model.encode(row['answer_llm'])

    evaluations.append(original_vector.dot(llm_vector))

300it [02:34,  1.94it/s]


In [23]:
np.percentile(evaluations,75)

np.float32(31.674305)

## Q3

In [33]:
def normalizer(vector):
    norm = np.sqrt((vector*vector).sum())
    v_norm = vector/norm

    return v_norm


In [36]:
nomalized = []

for idx, row in tqdm(df.iterrows()):
    original_vector = embedding_model.encode(row['answer_orig'])
    llm_vector = embedding_model.encode(row['answer_llm'])

    normalized_original = normalizer(original_vector)
    normalized_llm = normalizer(llm_vector)
    
    nomalized.append(normalized_original.dot(normalized_llm))

    

300it [02:38,  1.89it/s]


In [40]:
np.percentile(nomalized,75)

np.float32(0.8362348)

## Q4
#### Rouge
This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

Question - What's the F score for rouge-1?

In [45]:
test = df.iloc[10]

In [46]:
from rouge import Rouge

rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(test['answer_llm'], test['answer_orig'])[0]

In [49]:
scores['rouge-1']['f']

0.45454544954545456

## Q5

In [50]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [51]:
np.average([scores['rouge-1']['f'],scores['rouge-2']['f'],scores['rouge-l']['f']])

np.float64(0.35490034990035496)

## Q6

In [53]:

rouge_2_scores = []

for _,row in tqdm(df.iterrows()):
    
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]

    rouge_2_scores.append(scores['rouge-2']['f'])

300it [00:01, 221.16it/s]


In [54]:
np.average(rouge_2_scores)

np.float64(0.20696501983423318)